<a href="https://colab.research.google.com/github/kocurvik/mdrp/blob/main/demo/reposed_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RePosed Demo - Roma

This is a demo for the paper "RePoseD: Efficient Relative Pose Estimation With Known Depth Information" ICCV 2025 (oral) paper. DOI: TBA. Pre-print available on Arxiv: [2501.07742](https://arxiv.org/abs/2501.07742).

Code available in our [repo](https://github.com/kocurvik/mdrp). For more visual results see our [Project Page](https://kocurvik.github.io/reposed/).

The demo shows how to use the proposed solvers with MoGe and LightGlue to obtain the relative pose of two cameras and even merge the two MDE point clouds.

## Installing MoGe and LightGlue

First we install MoGe and Lightglue using their official repositories.

In [ ]:
# Install MoGe for depth estimation
!pip install git+https://github.com/microsoft/MoGe.git
# PR version, probably wont work now
!pip install https://github.com/kocurvik/mdrp/raw/refs/heads/main/demo/poselib-2.0.5-cp312-cp312-linux_x86_64.whl
# Install Roma matching
!pip install romatch[fused-local-corr]

### Installing PoseLib with Solvers

Next we install PoseLib. To use this demo you can use the precompiled wheel. However, if you use a different python config (OS/python version) you may need to compile the C++ code yourself using (this will take few minutes).

In [ ]:

### If you need to compile PoseLib in Colab yourself then use this:
#!git clone https://github.com/kocurvik/PoseLib
#%cd PoseLib
#!git checkout pr-mdrp
#!pip install pybind11_stubs
#!apt-get install libeigen3-dev
#!python setup.py install

### Outside of Google Colab you can try installing with:
#!pip install git+https://github.com/kocurvik/PoseLib@pr-mdrp

## Depth Estimation and Matching

We first process two images with MoGe and SuperPoint+LightGlue. We start by loading the models.

In [ ]:
import cv2
import torch
import numpy as np
from matplotlib import pyplot as plt

# from moge.model.v1 import MoGeModel
from moge.model.v2 import MoGeModel
from romatch import roma_indoor

import poselib

mde_model = MoGeModel.from_pretrained("Ruicheng/moge-2-vitl-normal").cuda()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
roma_model = roma_indoor(device=device)

Next we load the images. You can use the provided example images. You can load your own images here.

In [ ]:
!wget https://raw.githubusercontent.com/kocurvik/mdrp/refs/heads/main/demo/images/249.png
!wget https://raw.githubusercontent.com/kocurvik/mdrp/refs/heads/main/demo/images/450.png
im1 = "249.png"
im2 = "450.png"
cv_image1 = cv2.cvtColor(cv2.imread(im1), cv2.COLOR_BGR2RGB)
cv_image2 = cv2.cvtColor(cv2.imread(im2), cv2.COLOR_BGR2RGB)

# These images demonstrate why estimateting the scale of MDEs is important.
# !wget https://raw.githubusercontent.com/kocurvik/mdrp/refs/heads/main/demo/images/toy_car_1.jpg
# !wget https://raw.githubusercontent.com/kocurvik/mdrp/refs/heads/main/demo/images/toy_car_2.jpg
# cv_image1 = cv2.cvtColor(cv2.imread("toy_car_1.jpg"), cv2.COLOR_BGR2RGB)
# cv_image2 = cv2.cvtColor(cv2.imread("toy_car_2.jpg"), cv2.COLOR_BGR2RGB)

# !wget https://raw.githubusercontent.com/kocurvik/mdrp/refs/heads/main/demo/images/still_1.jpg
# !wget https://raw.githubusercontent.com/kocurvik/mdrp/refs/heads/main/demo/images/still_2.jpg
# cv_image1 = cv2.cvtColor(cv2.imread("still_1.jpg"), cv2.COLOR_BGR2RGB)
# cv_image2 = cv2.cvtColor(cv2.imread("still_2.jpg"), cv2.COLOR_BGR2RGB)

Then we run inference using the networks.

In [ ]:
# convert images to torch tensor
image1 = torch.tensor(cv_image1 / 255, dtype=torch.float32, device='cuda').permute(2, 0, 1)
image2 = torch.tensor(cv_image2 / 255, dtype=torch.float32, device='cuda').permute(2, 0, 1)

# Infer Moge
mde_out1 = mde_model.infer(image1)
mde_out2 = mde_model.infer(image2)



plt.title("Image 1")
plt.imshow(cv_image1)
plt.axis('off')
plt.show()

plt.title("Image 1 - depth")
plt.imshow(mde_out1['depth'].detach().cpu().numpy(), cmap='gray')
plt.axis('off')
plt.show()

plt.title("Image 2")
plt.imshow(cv_image2)
plt.axis('off')
plt.show()

plt.imshow(mde_out2['depth'].detach().cpu().numpy(), cmap='gray')
plt.axis('off')
plt.show()

# Extract Roma features
warp, certainty = roma_model.match(im1, im2, device=device)
# Sample matches for estimation
matches, certainty = roma_model.sample(warp, certainty)

H_A, W_A = cv_image1.shape[:2]
H_B, W_B = cv_image2.shape[:2]
# Convert to pixel coordinates (RoMa produces matches in [-1,1]x[-1,1])
kptsA, kptsB = roma_model.to_pixel_coordinates(matches, H_A, W_A, H_B, W_B)

We set up a function to visualize the pointcloud.

In [6]:
from typing import Optional
import plotly.graph_objects as go

# code taken from: https://github.com/cvg/Hierarchical-Localization/blob/master/hloc/utils/viz_3d.py

def init_figure(height: int = 800) -> go.Figure:
    """Initialize a 3D figure."""
    fig = go.Figure()
    axes = dict(
        visible=False,
        showbackground=False,
        showgrid=False,
        showline=False,
        showticklabels=True,
        autorange=True,
    )
    fig.update_layout(
        template="plotly_dark",
        height=height,
        scene_camera=dict(
            eye=dict(x=0., y=-.1, z=-2),
            up=dict(x=0, y=-1., z=0),
            projection=dict(type="orthographic")),
        scene=dict(
            xaxis=axes,
            yaxis=axes,
            zaxis=axes,
            aspectmode='data',
            dragmode='orbit',
        ),
        margin=dict(l=0, r=0, b=0, t=0, pad=0),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.1
        ),
    )
    return fig


def plot_points(
        fig: go.Figure,
        pts: np.ndarray,
        color: str = 'rgba(255, 0, 0, 1)',
        ps: int = 2,
        colorscale: Optional[str] = None,
        name: Optional[str] = None):
    """Plot a set of 3D points."""
    x, y, z = pts.T
    tr = go.Scatter3d(
        x=x, y=y, z=z, mode='markers', name=name, legendgroup=name,
        marker=dict(
            size=ps, color=color, line_width=0.0, colorscale=colorscale))
    fig.add_trace(tr)


def plot_interactive_pointcloud(xyz, rgb, subsampling_rate=0.05):
  # plots a plotly pointcloud
  # params:
  # xyz - n x 3 - array with 3D coordinates of points
  # rgb - n x 3 - array with RGB triplets in 0-255
  n = xyz.shape[0]

  l = np.random.rand(n) <= subsampling_rate
  fig = init_figure()
  plot_points(fig, xyz[l], color=rgb[l])
  fig.show()

# alternative viewer
# !pip install open3d
# import open3d as o3d
# def plot_interactive_pointcloud(xyz, rgb):
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(xyz)
#     pcd.colors = o3d.utility.Vector3dVector(rgb)  # Colors should be in [0,1]

#     # Visualize using Plotly
#     o3d.visualization.draw_plotly([pcd], point_sample_factor=0.05, window_name="Point Cloud Viewer", width=800, height=600)

Once depth and the intrinsics are available we can calculate the pointcloud.

In [7]:
def get_pointcloud(K_inv, depth_map, color_map):
  H, W = depth_map.shape
  u = np.arange(W)
  v = np.arange(H)
  uu, vv = np.meshgrid(u, v)

  z = depth_map.flatten()
  uu = uu.flatten()
  vv = vv.flatten()
  proj = np.vstack([uu, vv, np.ones_like(uu)])

  flattened_colors = color_map.reshape([-1, 3])

  return z[:, np.newaxis] * (K_inv @ proj).T, flattened_colors


### Running RePoseD

Finally, we can run the poselib implementation of RePoseD.

In [ ]:
# convert the matches to numpy
points1 = kptsA.cpu().numpy()
points2 = kptsB.cpu().numpy()

# extract depths for each keypoint
depth_map1 = mde_out1["depth"].cpu().numpy()
depth_map2 = mde_out2["depth"].cpu().numpy()
depths1 = depth_map1[points1[:, 1].astype(int), points1[:, 0].astype(int)]
depths2 = depth_map2[points2[:, 1].astype(int), points2[:, 0].astype(int)]

l = ~np.logical_and(np.isinf(depths1), np.isinf(depths2))

pp1 = np.array([cv_image1.shape[1] / 2, cv_image1.shape[0] / 2])
pp2 = np.array([cv_image2.shape[1] / 2, cv_image2.shape[0] / 2])

ransac_dict = {'max_epipolar_error': 2.0, 'max_reproj_error': 16.0}
# set this to true if you also want to estimate shit (calib case only)
ransac_dict['estimate_shift'] = False

# use this loss for better estimation
bundle_dict = {'loss_type': 'TRUNCATED_CAUCHY'}

h1, w1 = cv_image1.shape[:2]
h2, w2 = cv_image2.shape[:2]

# instrinsics1 = mde_out1['intrinsics'].cpu().numpy()
# instrinsics2 = mde_out2['intrinsics'].cpu().numpy()

# # MoGe also provides estimated focal length
# f1, px1, py1 = w1*instrinsics1[0, 0], w1*instrinsics1[0, 2], h1*instrinsics1[1, 2]
# f2, px2, py2 = w2*instrinsics2[0, 0], w2*instrinsics2[0, 2], h2*instrinsics2[1, 2]

# ## Since focals are known we can use the calibrated solver.
# camera1 = {'model': 'SIMPLE_PINHOLE', 'width': -1, 'height': -1, 'params': [f1, px1, py1]}
# camera2 = {'model': 'SIMPLE_PINHOLE', 'width': -1, 'height': -1, 'params': [f2, px2, py2]}
# pose, info = poselib.estimate_monodepth_pose(points1[l], points2[l], depths1[l], depths2[l], camera1, camera2, ransac_dict, bundle_dict)

## In case focals are not known or you don't trust them you can use this code to
## estimate them along with the pose with when the images are taken by a single
## camera.
image_pair, info = poselib.estimate_monodepth_shared_focal_pose(points1[l] - pp1, points2[l] - pp2, depths1[l], depths2[l], ransac_dict, bundle_dict)
f1 = image_pair.camera1.focal()
f2 = image_pair.camera2.focal()
pose = image_pair.pose

## If the images are captured by different cameras you can use this version.
# image_pair, info = poselib.estimate_monodepth_varying_focal_pose(points1[l] - pp1, points2[l] - pp2, depths1[l], depths2[l], ransac_dict, bundle_dict)
# f1 = image_pair.camera1.focal()
# f2 = image_pair.camera2.focal()
# pose = image_pair.pose

print(info)

K1 = np.array([[f1, 0, pp1[0]], [0, f1, pp1[1]],[0, 0, 1]])
K2 = np.array([[f2, 0, pp2[0]], [0, f2, pp2[1]],[0, 0, 1]])
K1_inv = np.linalg.inv(K1)
K2_inv = np.linalg.inv(K2)

xyz1, rgb1 = get_pointcloud(K1_inv, depth_map1, cv_image1)
xyz2, rgb2 = get_pointcloud(K2_inv, depth_map2, cv_image2)


# xyz1_in2 = (image_pair.pose.R @ xyz1.T).T + image_pair.pose.t
xyz1_in2 = (1/pose.scale) * ((pose.R @ xyz1.T).T + pose.t)

print("Estimated scale: ", pose.scale)

xyz_merged = np.vstack([xyz1_in2, xyz2])
rgb_merged = np.vstack([rgb1, rgb2])

plot_interactive_pointcloud(xyz_merged, rgb_merged)

# if you want a nicer pointcloud you can use
# note that this may cause lag or make Google Colab completely unresponsive
# plot_interactive_pointcloud(xyz_merged, rgb_merged, subsampling_rate=1.0)